# 7. 랜덤 포레스트

## 7.1 핵심 개념

**랜덤포레스트(random forest) 는 학습 데이터를 이용하여 여러 의사결정트리를 생성한 후, 해당 결과를 종합하여 결과를 도출하는 앙상블 기법**입니다. 즉, 랜덤포레스트는 의사결정 나무 수십~수백개가 예측한 분류 혹은 회귀분석 값의 평균을 구해내는 모델입니다. <u>학습 데이터는 무작위로 샘플링 되며 다수의 의사결정 트리로 분석 되기 때문에 랜덤 포레스트라는 이름이 붙었습니다.</u>

![랜덤포레스트](./extrafiles/randomforest.png)

**[데이터 샘플링 하는 과정]**

1. 데이터에서 부트스트래핑 과정을 통해 N개의 샘플링 데이터 셋을 생성  
   ㄴ 부트스트랩(핑) : 현재 상황에서 어떻게든 한다는 의미로 컴퓨터 용어로는 더 복잡한 도구를 만들수 있도록 도와주는 단순 도구 또는 그러한 작업을 지칭한다.
2. 각 샘플링된 데이터 셋에서 임의의 변수 선택. (sqrt(M)개 또는 M/3 개)
3. 의사결정트리들을 종합하여 앙상블 모델을 만들고 OOB Error 를 통해 오분류율을 평가  
   ㄴ OOB Error(Out-of-bag Error)  
      ㄴ OOB 데이터는 부트스트랩을 통한 데이터를 추출 했을때 train 데이터에 속하지 않은 데이터들을 의미.  
      ㄴ OOB Error 는 부트스트랩에 포함되지 않은 데이터들의 Decision Tree 처리결과 나온 예측값과 실제값의 차이를 의미.

## 7.2 scikit-learn

**랜덤 포레스트는 sklearn.ensemble 패키지**에 속해 있다. **분류 알고리즘으로는 RandomeForestClassifier**, **회귀분석 알고리즘으로는 RandomForestRegressor** 가 있다.

|sklearn.ensemble|Ensemble Methods|
|:--|:--|
|skleanr.ensemble.AdaBoostClassifier() |An AdaBoost classifier. |
|skleanr.ensemble.AdaBoostRegressor() |An AdaBoost regressor. |
|skleanr.ensemble.BaggingClassifier() |A Bagging classifier. |
|skleanr.ensemble.BaggingRegressor() |A Bagging regressor. |
|skleanr.ensemble.ExtraTreeClassifier() |An extra-tree classifier. |
|skleanr.ensemble.ExtraTreeRegressor() |An extra-tree regressor. |
|skleanr.ensemble.GradientBoostingClassifier() |Gradient Boosting for classification. |
|skleanr.ensemble.GradientBoostingRegressor() |Gradient Boosting for regressor.|
|skleanr.ensemble.IsolationForest() |Isolation Forest Algorithm. |
|skleanr.ensemble.RandomForestClassifier() |A random forest classifier. |
|skleanr.ensemble.RandomForestRegressor() |A random forest regressor. |
|skleanr.ensemble.RandomTreeEmbedding() |An ensemble of totally random trees.|
|skleanr.ensemble.StackingClassifier() |Stack of estimators with a final classifier.|
|skleanr.ensemble.StackingRegressor() |Stack of esimators with a final regressor. |

랜덤포레스의 분로와 회귀 기본 디폴트 옵션은 거의 동일합니다. 다만 의사 결정나무와 마찬가지로 **분류 모델의 경우 기준(criterion)은 'gini' 이고 회귀 분석 모델의 기준은 mse** 가 됩니다.

|Hyper Parameter||
|:--|:--|
|n_estimators|랜덤포레스트를 구성하는 의사경정트리의 수 (기본값:100|
|max_features|의사결정트리에 적용되는 특성변수의 수<br/>- "auto"/"sqrt" : sqrt(n_features)<br/>- "log2" : log2(n_features)<br/>- "none" : n_features|

## 7.3 분석 코드

### Part1. 분류(Classification)

In [1]:
# 경고레벨조정
import warnings
warnings.filterwarnings("ignore")

# 데이터 로드
import pandas as pd
data = pd.read_csv("./extrafiles/breast-cancer-wisconsin.csv", encoding='utf-8')

# 컬럼정보 확인
print(data.columns)

# 독립변수/ 종속변수 분리
X = data[['Clump_Thickness', 'Cell_Size', 'Cell_Shape',
       'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
       'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses']]
y = data[['Class']]

# train-test data 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# stratify 효과 - 범주형 변수를 유사한 비율로 train / test 데이터로 분리시켜 준다.
print(y_train.mean())
print(y_test.mean())

# 표준화 작업 - MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

Index(['code', 'Clump_Thickness', 'Cell_Size', 'Cell_Shape',
       'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
       'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses', 'Class'],
      dtype='object')
Class    0.349609
dtype: float64
Class    0.350877
dtype: float64


In [3]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

1.0

In [4]:
# 훈련데이터의 혼동행렬 작성
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print("훈련데이터 오차행렬 : \n", confusion_train)

훈련데이터 오차행렬 : 
 [[333   0]
 [  0 179]]


In [5]:
# 훈련데이터의 분류 레포트 작성
from sklearn.metrics import classification_report
cfreport_train = classification_report(y_train, pred_train)
print("분류예측 레포트 : \n", cfreport_train)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       333
           1       1.00      1.00      1.00       179

    accuracy                           1.00       512
   macro avg       1.00      1.00      1.00       512
weighted avg       1.00      1.00      1.00       512



In [6]:
# 테스트데이터 예측결과 생성
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.9649122807017544

In [7]:
# 테스트데이터의 혼동행렬 작성
confusion_test = confusion_matrix(y_test, pred_test)
print("테스트데이터 오차행렬 : \n", confusion_test)

테스트데이터 오차행렬 : 
 [[106   5]
 [  1  59]]


In [8]:
# 테스트데이터의 분류 레포트 작성
from sklearn.metrics import classification_report
cfreport_test = classification_report(y_test, pred_test)
print("분류예측 레포트 : \n", cfreport_test)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.99      0.95      0.97       111
           1       0.92      0.98      0.95        60

    accuracy                           0.96       171
   macro avg       0.96      0.97      0.96       171
weighted avg       0.97      0.96      0.97       171



의사결정트리와는 달리 훈련용 데이터 테스트 데이터 모두 높은 예측율을 보여주고 있다. 즉 과적합 되지는 않았다는 의미이다.

In [10]:
# 하이퍼 파라미터 튜닝 - Grid Search
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators' :range(100, 1000, 100), 
              'max_features' :['auto', 'sqrt', 'log2']}
             
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': range(100, 1000, 100)})

In [12]:
# 파라미터 튜닝 결과 확인
print("Best Parameter : {}".format(grid_search.best_params_))
print("Best Score : {:.4f}".format(grid_search.best_score_))
print("Test set Score : {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter : {'max_features': 'sqrt', 'n_estimators': 200}
Best Score : 0.9746
Test set Score : 0.9649


In [13]:
# 하이퍼 파라미터 튜닝2 - Randomized Search
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'n_estimators' :randint(low=100, high=1000), 
                  'max_features' :['auto', 'sqrt', 'log2']}

random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_distribs, n_iter=20, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=20,
                   param_distributions={'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002134B4BF430>})

In [14]:
# 파라미터 튜닝 결과값 확인
print("Best Parameter : {}".format(random_search.best_params_))
print("Best Score : {:.4f}".format(random_search.best_score_))
print("Test set Score : {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter : {'max_features': 'log2', 'n_estimators': 599}
Best Score : 0.9746
Test set Score : 0.9649


### Part2. 회귀(Regression)

In [15]:
# 데이터 로드
data2 = pd.read_csv('./extrafiles/house_price.csv', encoding='utf-8')

print(data2.columns)

X = data2[data2.columns[1:5]]
y = data2[['house_value']]

print(X.columns)

# train-test data 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# stratify 효과 - 범주형 변수를 유사한 비율로 train / test 데이터로 분리시켜 준다.
print(y_train.mean())
print(y_test.mean())

# 표준화 작업 - MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

Index(['housing_age', 'income', 'bedrooms', 'households', 'rooms',
       'house_value'],
      dtype='object')
Index(['income', 'bedrooms', 'households', 'rooms'], dtype='object')
house_value    189260.967812
dtype: float64
house_value    188391.001357
dtype: float64


In [16]:
# 의사결정트리 모델 적용
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.9380740281242856

In [17]:
# 테스트 데이터 모델 적용
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.5841233859621466

In [18]:
# 회귀분석의 지표 R Square n RMSE
# RMSE (Root Mean Squared Error)
from sklearn.metrics import mean_squared_error
import numpy as np
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈  련데이터 RMSE:", np.sqrt(MSE_train))
print("테스트데이터 RMSE:", np.sqrt(MSE_test))

훈  련데이터 RMSE: 23751.30406433138
테스트데이터 RMSE: 61651.652749843626


In [19]:
# 하이퍼 파라미터 튜닝 - Grid Search
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators' :range(100, 500, 100), 
              'max_features' :['auto', 'sqrt', 'log2']}
grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': range(100, 500, 100)})

In [20]:
# 파라미터 튜닝 결과 확인
print("Best Parameter : {}".format(grid_search.best_params_))
print("Best Score : {:.4f}".format(grid_search.best_score_))
print("Test set Score : {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter : {'max_features': 'log2', 'n_estimators': 400}
Best Score : 0.5687
Test set Score : 0.5933


In [21]:
# 하이퍼 파라미터 튜닝2 - Randomized Search
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'n_estimators' :randint(low=100, high=500), 
                  'max_features' :['auto', 'sqrt', 'log2']}

random_search = RandomizedSearchCV(RandomForestRegressor(), param_distributions=param_distribs, n_iter=20, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=20,
                   param_distributions={'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002134C569F40>})

In [22]:
# 파라미터 튜닝 결과값 확인
print("Best Parameter : {}".format(random_search.best_params_))
print("Best Score : {:.4f}".format(random_search.best_score_))
print("Test set Score : {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter : {'max_features': 'sqrt', 'n_estimators': 439}
Best Score : 0.5688
Test set Score : 0.5930


**[종합정리]**

**랜덤포레스트의 경우 하이퍼 파라미터가 튜닝되지 않은 상태에서는 과적합 경향**을 보였습니다만, <u>적절한 모델수와 특성치를 탐색한 결과는 좋은 결과를 나타내고 있으며, 다른 앙상블 기법들보다 단순한 구조이면서 강력한 성능</u>을 보입니다. 때문에 실전 분석에서 가장 많이 사용되는 알고리즘중 하나가 랜덤 포레스트 입니다.